# Swap pre-generated climate TS into a Source model and apply to storages

This notebook inputs pre-generated climate timeseries into a Source model and applies it to the storages. You will need to open up the model you want to update and use the Web Server Monitoring tool to specify a port and allow scripts first. The port number should be the same as the one specified in the code below. 

Pre-generated climate TS can be produced using the Climate_TS_production tool (which will also apply any scaling factors required). 

The script assumes that entirely new TS will be swapped (i.e. does not extend existing TS). This script was written to retain metaparameters in "Hydro only" models being extended for climate data so that the In-built calibration tool can be accessed. 

Note: The DS plugin climate collation is still accessible, but if using this plugin to extend climate data, metaparameters won't be accessible in the via the in-built calibration tool as the plugin "removes zero usage timesereis". 

In [1]:
%matplotlib inline
import veneer
#from veneer.actions import switch_data_source
import climate_utils
import shapely
from climate_utils import zonal
import rasterio
import os
import string
from glob import glob
import numpy as np
import pandas as pd
import sys
from datetime import datetime,timedelta
import requests
import json

In [2]:
# Defines the switch_data_source veneer function (the bugs in latest veneer should be fixed. Code below is a work around)

def switch_data_source(v,from_set,to_set,remove_original=True):
    switch_input_sets_script="""
%s
dm = scenario.Network.DataManager
orig_item = dm.DataGroups.First(lambda g: g.Name=='%s')
dest_item = dm.DataGroups.First(lambda g: g.Name=='%s')

def transfer_usages(orig_item,dest_item):
  for orig_detail in orig_item.DataDetails:
    dest_detail = dest_item.DataDetails.First(lambda dd:dd.Name==orig_detail.Name)

    for usage in orig_detail.Usages:
      dest_detail.Usages.Add(usage)
    orig_detail.Usages.Clear();

transfer_usages(orig_item,dest_item)

"""%(v.model._init_script(),from_set,to_set)
    if remove_original:
        switch_input_sets_script += "\ndm.RemoveGroup(orig_item)"
    return v.model.run_script(switch_input_sets_script)

## Settings


In [3]:
# Information about the model structure
DATA_SOURCE='Climate Data'                      # Where the climate data is stored within Source
EXISTING_DATA_SOURCE=DATA_SOURCE
NEW_DATA_SOURCE='New Climate Data'         # Where it will be stored after this notebook (needs to be different)
# NAME_ATTRIBUTE='Catchmt'

DATA_SOURCE_NAMING=string.Template('${variable} for ${catchment}') # How the columns are named


CRS='epsg:3577'   # Australian Albers
PORT=10000 # Where Veneer is running 

STORAGE_TO_CATCHMENT_MAPPING='MW_storage_climate_sources.csv'


In [4]:
STORAGE_CLIMATE_SOURCES=pd.read_csv(STORAGE_TO_CATCHMENT_MAPPING)

In [5]:
v = veneer.Veneer(port=PORT)


## Get the pre-produced timeseries

In [6]:
NEW_CLIMATE_PATH='D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW'
New_climate_file_path = os.path.join(NEW_CLIMATE_PATH,'*.csv')

In [7]:
import glob
New_climate_filenames = glob.glob(New_climate_file_path)
New_climate_filenames

['D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #1.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #10.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #100.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #101.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #102.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #103.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #104.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #105.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #106.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #107.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #108.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC #109.csv',
 'D:\\Climate_extension\\outputTS\\Jan1970_Mar2018\\MW\\pet for SC 

In [8]:
appended_data=[]

for f in New_climate_filenames:
    data = pd.read_csv(f,index_col=0, header=0)
    appended_data.append(data)

new_climate = pd.concat(appended_data, axis=1)

new_climate

,pet for SC #1,pet for SC #10,pet for SC #100,pet for SC #101,pet for SC #102,pet for SC #103,pet for SC #104,pet for SC #105,pet for SC #106,pet for SC #107,...,rainfall for SC #90202,rainfall for SC #91,rainfall for SC #92,rainfall for SC #93,rainfall for SC #94,rainfall for SC #95,rainfall for SC #96,rainfall for SC #97,rainfall for SC #98,rainfall for SC #99
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-01,7.053084,6.769551,7.633924,7.700000,7.697796,7.521442,7.651593,7.700000,7.700000,7.697650,...,0.000000,6.301346,11.486364,11.478633,3.972530,0.000000,0.224059,0.000000,0.000000,0.0
1970-01-02,7.304993,7.009407,6.900063,6.900000,7.100000,7.058462,7.100000,7.100000,7.100000,7.053189,...,0.611618,4.288313,5.708675,7.763635,6.723997,6.935779,6.102472,0.475344,1.462758,0.7
1970-01-03,6.701248,6.502726,6.933260,6.968774,7.033236,7.094864,7.106795,7.032266,6.953684,7.064851,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1970-01-04,7.201248,6.890558,7.271346,7.237547,7.300041,7.258462,7.300000,7.300000,7.300000,7.299217,...,0.839501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1970-01-05,7.600000,7.537445,7.700000,7.700000,7.700000,7.699651,7.762763,7.700000,7.700000,7.699217,...,0.252675,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1970-01-06,8.301248,8.071660,7.066740,7.031226,7.100000,7.151149,7.100000,7.100000,7.100000,7.100392,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1970-01-07,7.994383,7.512624,7.600000,7.600000,7.615012,7.558462,7.575797,7.600000,7.600000,7.598825,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1970-01-08,7.901248,7.450093,7.500000,7.500000,7.498898,7.436360,7.475797,7.500000,7.500000,7.499217,...,0.199327,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1970-01-09,7.853084,7.509837,7.576972,7.600000,7.531997,7.304342,7.303214,7.300000,7.300000,7.299608,...,0.575114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


## Retrieve the existing climate data from Source...


In [12]:
data_sources = v.data_sources()
df = data_sources.as_dataframe()
df[df.Name==EXISTING_DATA_SOURCE]

,FullName,Items,Name,Path,id
12,/Climate Data,"[{'Details': [{'Name': 'rainfall for SC #158',...",Climate Data,/,/dataSources/Climate Data


In [13]:
veneer.general.PRINT_URLS=True
%time existing_climate_data_record = v.data_source(EXISTING_DATA_SOURCE)
existing_climate_data = existing_climate_data_record['Items'][0]['Details']

*** /dataSources/Climate Data - /dataSources/Climate Data ***
Wall time: 16.5 s


## Push the new climate data up to Source as a new Data Source



In [14]:
v.create_data_source(NEW_DATA_SOURCE,new_climate,units='mm/day')


(200, None)

In [15]:
switch_data_source(v,EXISTING_DATA_SOURCE,NEW_DATA_SOURCE)

{'Exception': None, 'Response': None, 'StandardError': '', 'StandardOut': ''}

## Apply to storage...

At this point, the new data is loaded in Source and applied to all the subcatchments.

The data isn't hooked up to the storages yet, but will be assigned in the following cells.


In [16]:
STORAGE_CLIMATE_SOURCES

,Storage,SC
0,dam_PeterFaust,146
1,dam_Teemburra,150
2,weir_Dumbleton,75
3,weir_Marian,153
4,weir_Mirani,88


In [17]:
veneer.general.PRINT_SCRIPTS=False
for (_,row) in STORAGE_CLIMATE_SOURCES.iterrows():
    v.model.node.assign_time_series(
        'RainfallInMetresPerSecond',
        'rainfall for SC #%s'%row.SC,
        NEW_DATA_SOURCE,
        nodes=row.Storage)
    v.model.node.assign_time_series(
        'EvaporationInMetresPerSecond',
        'pet for SC #%s'%row.SC,
        NEW_DATA_SOURCE,
        nodes=row.Storage)
    

## DONE.

Ok, now the new data is loaded in Source, the data is attached to the relevant subcatchments AND storages.

### NOTE: The old data sources still exist in Source - at the moment you need to delete them manually...



In [18]:
v.drop_all_runs()

*** /runs - /runs ***
